In [ ]:
!pip install pyrsgis
!pip install pyswarm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pyswarm: filename=pyswarm-0.6-py3-none-any.whl size=4478 sha256=62ce6be75cf906dba5dfffd8165718330f89cd8aacafed4563003dd7fa65a375
  Stored in directory: /root/.cache/pip/wheels/ff/d2/b7/80118e5698de2bd0b8d1b3397abf7fdfc45c15ffc454b52145
Successfully built pyswarm


In [ ]:
import os, math, numpy as np
from pyrsgis import raster, convert
from osgeo import gdal
from copy import deepcopy
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt
from pyswarm import pso
from google.colab import drive

drive.mount('/content/drive')
f2017 = "/content/drive/MyDrive/Capstone_Data/Bangalore_data/2017.tif"
f2020 = "/content/drive/MyDrive/Capstone_Data/Bangalore_data/2020.tif"
road_network_path = "/content/drive/MyDrive/Capstone_Data/Bangalore_data/road.tif"
pop = "/content/drive/MyDrive/Capstone_Data/Bangalore_data/pop.tif"
metro = "/content/drive/MyDrive/Capstone_Data/Bangalore_data/metro.tif"

# Read the input rasters
road_ds,road_arr = raster.read(road_network_path,bands=1)
ds1, arr1 = raster.read(f2017,bands=1)
ds2,arr2 = raster.read(f2020,bands=1)
pop_ds, pop_arr = raster.read(pop,bands=1)
metro_ds,metro_arr = raster.read(metro,bands=1)

no_rows = ds1.RasterYSize
no_cols = ds1.RasterXSize
kernelSize = 3
predicted_arr = deepcopy(arr1)
sideMargin = math.ceil(kernelSize/2)

Mounted at /content/drive


In [ ]:
def fitness_function(parameters):

  builtUpThreshold = parameters[0]
  distFromRoad = parameters[1]
  popThreshold = parameters[2]
  metroThreshold = parameters[3]

  for y in range(sideMargin,no_rows-(sideMargin-1)):
      for x in range(sideMargin,no_cols-(sideMargin-1)):
          kernel = arr1[y-(sideMargin-1):y+(sideMargin), x-(sideMargin-1):x+(sideMargin)]
          builtupCount = sum(sum(kernel==1))
          if (builtupCount >= builtUpThreshold and road_arr[y,x]<=distFromRoad and arr1[y,x]!=3 and pop_arr[y,x]>=popThreshold and metro_arr[y,x]>=metroThreshold):
              predicted_arr[y,x]=1

  ssim_score = ssim(arr2, predicted_arr,data_range=4)
  return -ssim_score  # Return the negative value for minimization


lb = [1,100,-100,0]
ub = [9,300,600,900]

xopt, fopt = pso(fitness_function,lb,ub, swarmsize=30, maxiter=10)

Stopping search: maximum iterations reached --> 10


In [ ]:
print(xopt,-fopt)

[  7.00027176 139.08590463 209.35314159 163.04706401] 0.5416009530800527


In [ ]:
builtUpThreshold = 7
distFromRoad = 140
popThreshold = 210
metroThreshold = 163

for y in range(sideMargin,no_rows-(sideMargin-1)):
    for x in range(sideMargin,no_cols-(sideMargin-1)):
        kernel = arr1[y-(sideMargin-1):y+(sideMargin), x-(sideMargin-1):x+(sideMargin)]
        builtupCount = sum(sum(kernel==1))
        if (builtupCount >= builtUpThreshold and road_arr[y,x]<=distFromRoad and arr1[y,x]!=3):
            predicted_arr[y,x]=1

def exportPredicted(outFileName):
    driver = gdal.GetDriverByName("GTiff")
    outdata = driver.Create(outFileName, no_cols, no_rows, 1, gdal.GDT_UInt16) # option: GDT_UInt16, GDT_Float32
    outdata.SetGeoTransform(ds1.GetGeoTransform())
    outdata.SetProjection(ds1.GetProjection())
    outdata.GetRasterBand(1).WriteArray(predicted_arr)
    outdata.GetRasterBand(1).SetNoDataValue(0)
    outdata.FlushCache()
    outdata = None

print("SSIM:", ssim(arr2,predicted_arr,data_range=4))
exportPredicted("Bangalore_PSO_2020.tif")

SSIM: 0.5192381068392012
